In [0]:
%sql
-- EDA

In [0]:
%sql
WITH dataset_summary AS (
    SELECT 
        COUNT(*) AS total_registros_hora_celda,
        SUM(target) AS total_eventos_incendio,
        COUNT(DISTINCT grid_id) AS total_puntos_monitoreados,
        MIN(timestamp_clima) AS fecha_inicio,
        MAX(timestamp_clima) AS fecha_fin,
        ROUND(SUM(target) * 100.0 / COUNT(*), 4) AS tasa_prevalencia_fuego
    FROM fire_risk_project.03_gold.gold_dataset_full
)
SELECT * FROM dataset_summary;

In [0]:
%sql
SELECT 
    CASE 
        WHEN fire_confidence = 3 THEN 'Alta (High)'
        WHEN fire_confidence = 2 THEN 'Nominal'
        WHEN fire_confidence = 1 THEN 'Baja (Low)'
        ELSE 'Desconocido'
    END AS nivel_confianza,
    COUNT(*) AS cantidad_incendios,
    ROUND(AVG(fire_intensity), 2) AS intensidad_promedio_frp,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) AS porcentaje_del_total
FROM fire_risk_project.03_gold.gold_dataset_full
WHERE target = 1
GROUP BY fire_confidence
ORDER BY cantidad_incendios DESC;

In [0]:
%sql
WITH monthly_trends AS (
    SELECT 
        MONTH(timestamp_clima) AS mes,
        SUM(target) AS total_incendios,
        ROUND(AVG(temperature_2m), 1) AS temp_promedio_mes,
        ROUND(AVG(precipitation), 2) AS lluvia_promedio_mes
    FROM fire_risk_project.03_gold.gold_dataset_full
    GROUP BY MONTH(timestamp_clima)
)
SELECT 
    mes, 
    total_incendios,
    temp_promedio_mes,
    RANK() OVER (ORDER BY total_incendios DESC) as ranking_peligrosidad
FROM monthly_trends
ORDER BY mes;

In [0]:
%sql
WITH fire_context AS (
    SELECT 
        grid_id,
        timestamp_clima,
        temperature_2m,
        wind_speed_10m,
        CASE 
            WHEN temperature_2m > 30 AND wind_speed_10m > 20 THEN 'Extremo (Calor+Viento)'
            WHEN temperature_2m > 30 THEN 'Ola de Calor'
            WHEN wind_speed_10m > 20 THEN 'Viento Fuerte'
            WHEN relative_humidity_2m < 30 THEN 'Sequía Extrema'
            ELSE 'Normal / Otro'
        END AS segmento_climatico
    FROM fire_risk_project.03_gold.gold_dataset_full
    WHERE target = 1
)
SELECT 
    segmento_climatico,
    COUNT(*) AS cantidad_incendios,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM fire_context), 2) AS porcentaje_incendios
FROM fire_context
GROUP BY segmento_climatico
ORDER BY cantidad_incendios DESC;

In [0]:
%sql
SELECT 
    grid_id,
    timestamp_clima,
    fire_intensity AS FRP_Potencia, 
    temperature_2m AS Temp_C,
    wind_speed_10m AS Viento_kmh,
    relative_humidity_2m AS Humedad_pct,
    RANK() OVER (ORDER BY fire_intensity DESC) AS ranking_intensidad
FROM fire_risk_project.03_gold.gold_dataset_full
WHERE target = 1
LIMIT 10;

In [0]:
%sql
SELECT 
    grid_id,
    latitude,
    longitude,
    SUM(target) AS total_incendios, 
    MAX(fire_intensity) AS intensidad_maxima_registrada,
    ROUND(AVG(temperature_2m), 1) AS temp_promedio_zona,
    ROUND(SUM(target) * 100.0 / SUM(SUM(target)) OVER(), 2) AS porcentaje_del_total_global
FROM fire_risk_project.03_gold.gold_dataset_full
GROUP BY grid_id, latitude, longitude
ORDER BY total_incendios DESC
LIMIT 15;

In [0]:
%sql
WITH anual_metrics AS (
    SELECT 
        YEAR(timestamp_clima) AS anio,
        COUNT(*) AS horas_monitoreadas,
        SUM(target) AS total_incendios,
        AVG(fire_intensity) AS intensidad_promedio
    FROM fire_risk_project.03_gold.gold_dataset_full
    GROUP BY YEAR(timestamp_clima)
)
SELECT 
    anio,
    total_incendios AS incendios_este_anio,
    LAG(total_incendios) OVER (ORDER BY anio) AS incendios_anio_anterior,
    total_incendios - LAG(total_incendios) OVER (ORDER BY anio) AS diferencia_neta,
    ROUND(intensidad_promedio, 2) AS intensidad_avg
FROM anual_metrics
ORDER BY anio;